## 1. Daten sammeln

Beim Daten sammeln wird die Bibliothek `requests` verwendet, um auf die Schnittstelle von Smartvote zuzugreifen. Für diese Schnittstelle habe ich keine Dokumentation gefunden, sie kann aber relativ einfach [reverse-engineert](https://de.wikipedia.org/wiki/Reverse_Engineering) werden. Das Reverse-Engineering ist hier nicht dokumentiert.

Als erstes müssen die verwendeten Libraries importiert werden.
- `requests` für das Anfordern der Daten
- `json` für das Handhaben der gesammelten Daten


In [1]:
import requests
import json

Hier kann eingestellt werden, welche Daten gesammelt werden sollen - 1 für Nationalratswahlen, 2 für Ständeratswahlen.

In [2]:
mode = 1
# 1 = Nationalratswahlen
# 2 = Ständeratswahlen

electionId = "1057" if mode == 1 else "1084"
filename = "nationalrat_raw.json" if mode == 1 else "ständerat_raw.json"

Es folgt das Aufrufen der Schnittstelle zum tatsächlichen Holen der Daten.

In [3]:
url = "https://backend.smartvote.ch/api/graphql"
headers = {}

body = {
    "operationName": "SearchCandidates",
    "variables": {"searchParams": {"electionId": electionId}},
    "query": """
    query SearchCandidates($searchParams: CandidateSearchParams!) {
        candidates(searchParams: $searchParams) {
            ...SearchCandidate
        }
    }

    fragment SearchCandidate on Candidate {
        id
        firstname
        lastname
        yearOfBirth
        profileImageUrl
        isIncumbent
        isElected
        partyAbbreviation
        partyColor
        hasSmartvoteProfile
        gender

        answers {
            ...Answer
        }
    }

    fragment Answer on Answer {
        questionId
        value
    }
    """,
}


response = requests.post(url, headers=headers, json=body)


if response.status_code == 200:
    candidates = response.json()["data"]["candidates"]
    print(f"Request was successful and returned {len(candidates)} candidate(s)")
else:
    print(f"Request failed with status code {response.status_code}")

Request was successful and returned 5925 candidate(s)


Ein Beispiel für eine Kandidatin.

In [4]:
print(json.dumps(candidates[242 if len(candidates) > 242 else 52], indent=4, ensure_ascii=False))

{
    "id": "55096",
    "firstname": "Jacqueline",
    "lastname": "Badran",
    "yearOfBirth": 1961,
    "profileImageUrl": "https://backend.smartvote.ch/rails/active_storage/blobs/redirect/eyJfcmFpbHMiOnsiZGF0YSI6NTEyNDcsInB1ciI6ImJsb2JfaWQifX0=--552ccdeca113cff802db056e6b4d524d9778558d/image.jpg",
    "isIncumbent": true,
    "isElected": true,
    "partyAbbreviation": "SP",
    "partyColor": "#F0554D",
    "hasSmartvoteProfile": true,
    "gender": "f",
    "answers": [
        {
            "questionId": "32214",
            "value": 0
        },
        {
            "questionId": "32218",
            "value": 100
        },
        {
            "questionId": "32222",
            "value": 0
        },
        {
            "questionId": "32223",
            "value": 0
        },
        {
            "questionId": "32216",
            "value": 100
        },
        {
            "questionId": "32224",
            "value": 75
        },
        {
            "questionId": "3223

Als nächstes sortieren wir alle Kandidat*innen aus, die kein Smartvote-Profil haben.

In [5]:
candidates = [candidate for candidate in candidates if candidate["hasSmartvoteProfile"]]

Hier werden die gesammelten Daten in eine JSON-Datei gespeichert.

In [6]:
# Write the response data to a file
with open(filename, "w") as json_file:
    json.dump(candidates, json_file, indent=4, ensure_ascii=False)

print(f"Response has been written to {filename}.")

Response has been written to nationalrat_raw.json.
